In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
import WUBRG
from WUBRG import COLOR_ALIASES_SUPPORT, COLOR_ALIASES, COLOUR_GROUPINGS, COLOUR_GROUPS

import set_info
from set_info import SETS, FORMATS, STAT_NAMES
DATA_CACHE = None
PANDAS_CACHE = None
print(f'SETS:          {str(SETS)}')
print(f'FORMATS:       {str(FORMATS)}')

WRANING: 'discord' failed to import. Emoji support unavailable
SETS:          ['MID']
FORMATS:       ['PremierDraft', 'TradDraft']


In [18]:
# Filters out cards that have a number of games played less than p% of # GP.
def min_play_filter(df, p):
    min_game = df['# GP'].mean() * (p/100)
    return df[df['# GP'] >= min_game]

# Filters out cards that don't fall within the colour identy.
def color_id_filter(df, color_string):
    if color_string is not None and color_string != '':
        return df[df['Color'].apply(lambda x: set(x) <= set(color_string))]
    else:
        return df

# Filters out cards that aren't in the given rarities.
def rarity_filter(df, rarity):
    if rarity is not None and rarity != '':
        return df[df['Rarity'].apply(lambda x: set(x) <= set(rarity))]
    else:
        return df

# Gets the top n cards, based on a particular stat column.
# Can filter based on card colours, rarity.
# Can get the bottom n cards with 'reverse=True'
def get_top(df, col, n=5, card_colors=None, card_rarity=None, min_thresh=10, reverse=False):
    filtered = min_play_filter(df, min_thresh)
    filtered = color_id_filter(filtered, card_colors)
    filtered = rarity_filter(filtered, card_rarity)

    # Functional XOR.
    reverse = reverse != (col == "ALSA" or col == "ATA")
    
    # Return the smallest values if we're dealing with pick orders
    if reverse != (col == "ALSA" or col == "ATA"):
        return filtered.nsmallest(n, col)
    else:
        return filtered.nlargest(n, col)

# Gets a suite of data about the given colour's position in the metagame.
def get_color_metadata(s, f, color, stat='GIH WR'):   
    top_commons = get_top(PANDAS_CACHE[s][f][''], stat, n=10, card_colors=color, card_rarity='C')
    top_commons.columns.name = f'Top 10 Commons'
    top_uncommons = get_top(PANDAS_CACHE[s][f][''], stat, n=10, card_colors=color, card_rarity='U')
    top_uncommons.columns.name = f'Top 10 Uncommons'
    
    # TODO: Generate four colour pairs containing the main color
    color_pairs = ['UW', 'UB', 'UR', 'UG']
    color_pair_dfs = dict()
    
    for col in color_pairs:
        c = WUBRG.get_color_identity(col)
        commons = get_top(PANDAS_CACHE[s][f][c], stat, n=5, card_rarity='C')
        uncommons = get_top(PANDAS_CACHE[s][f][c], stat, n=5, card_rarity='U')
        df = pd.concat([commons, uncommons])
        df.columns.name = f'{col} Top Cards'
        color_pair_dfs[c] = df
        
    # TODO: Add in a fun fact here.
        
    return (top_commons, top_uncommons, color_pair_dfs)

In [4]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_qudrant_cards_df(df, rarity, iwd_thresh, func1, func2, cols=None):
    by_rarity = df[df['Rarity'] == rarity]
    games_played_mean = by_rarity['# GP'].mean()
    cards = by_rarity[func1(by_rarity['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    if cols is not None:
        cards = cards[cols]
    return cards


def get_trap_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, gt, cols)

def get_trap_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_trap_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_niche_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, lt, cols)

def get_niche_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_niche_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_staple_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, gt, cols)

def get_staple_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_staple_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_dreg_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, lt, cols)

def get_dreg_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_dreg_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)

In [5]:
def filter_df(df, rarity, color):
    temp = df[df['Rarity'] == rarity]
    temp = temp[temp['Color'] == color]
    return temp

In [6]:
set_info.init_cache()
DATA_CACHE = set_info.DATA_CACHE
PANDAS_CACHE = set_info.PANDAS_CACHE

Checking for new format data...
MID PremierDraft data is up to date!
MID TradDraft data is up to date!
Done checking for new format data.

Checking for cache updates...
Parsing MID_PremierDraft.json...
MID PremierDraft added to DATA_CACHE!
Parsing MID_TradDraft.json...
MID TradDraft added to DATA_CACHE!
Done checking for cache updates.



In [7]:
premier = PANDAS_CACHE['MID']['PremierDraft']['WU'].sort_values('GP WR', ascending=False)
premier

test = premier[premier['Color'].apply(lambda x: set(x) <= set('WU'))]
#frame['Color'].apply(lambda x: any([k in x for k in 'WU']))
min_game = test['# GP'].mean() * 0.1
test[test['# GP'] >= min_game]

,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD
Poppet Stitcher,U,M,1408,1.396,1058,1.384,678,62.832,115,64.348,234,63.248,349,63.610,350,62.857,0.753
Spectral Adversary,U,M,1522,1.507,1115,1.507,1350,62.148,208,60.096,464,66.164,672,64.286,701,59.914,4.371
Grafted Identity,U,R,3522,1.755,2483,1.739,2901,61.668,510,63.725,933,66.238,1443,65.350,1477,58.158,7.192
"Adeline, Resplendent Cathar",W,R,3035,1.516,1906,1.517,2605,61.459,488,66.189,968,65.393,1456,65.659,1249,57.246,8.414
"Dennick, Pious Apprentice",WU,R,4968,2.604,2061,2.542,6192,61.337,1145,68.559,2087,64.494,3232,65.934,3205,57.254,8.680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bereaved Survivor,W,U,22911,3.675,3870,5.013,2828,53.571,512,50.391,950,51.684,1462,51.231,1443,55.371,-4.140
Sigarda's Splendor,W,R,5298,2.650,1631,2.841,1545,52.039,262,48.855,493,55.172,755,52.980,803,51.432,1.548
Cathar's Call,W,U,27435,4.570,2754,7.168,1052,51.521,177,52.542,345,51.884,522,52.107,553,51.356,0.751
Component Collector,U,C,112696,7.769,15330,11.067,1966,51.475,378,49.206,718,52.368,1096,51.277,983,51.984,-0.706


In [8]:
def center(df, stat):
    temp = df.copy()
    
    stat_mean = temp[stat].mean()
    temp[stat] = temp[stat] - stat_mean
    return temp.sort_values(stat, ascending=(stat == "ALSA" or stat == "ATA"))

In [9]:
center(PANDAS_CACHE['MID']['PremierDraft']['WU'], 'ALSA')

,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD
Wrenn and Seven,G,M,1217,-3.25136,931,1.119,18,50.000,1,0.000,5,40.000,6,33.333,12,58.333,-25.000
"Jerren, Corrupted Bishop",B,M,1358,-3.17736,1037,1.201,34,41.176,7,28.571,8,62.500,15,46.667,19,36.842,9.825
"Arlinn, the Pack's Hope",RG,M,1214,-3.17036,797,1.199,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0.000
The Meathook Massacre,B,M,1294,-3.16836,1062,1.240,25,52.000,6,66.667,11,63.636,17,64.706,9,33.333,31.373
Lord of the Forsaken,B,M,1280,-3.16036,1007,1.226,40,62.500,8,50.000,10,70.000,18,61.111,23,65.217,-4.106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tapping at the Window,G,C,114388,3.89864,12518,11.960,4,75.000,0,NaN,2,50.000,2,50.000,2,100.000,-50.000
Abandon the Post,R,C,115976,3.92664,12710,11.962,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0.000
Thraben Exorcism,W,C,115964,4.14964,12221,12.469,276,51.449,45,35.556,77,44.156,122,40.984,157,59.236,-18.252
Return to Nature,G,C,116778,4.51964,11465,12.999,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0.000


In [10]:
stat = 'GIH WR'
x = get_top(PANDAS_CACHE['MID']['PremierDraft']['WU'], stat, n=5, card_colors=None, card_rarity='MR')
["Color", "ALSA", "# GP", "GP WR", "# GIH", "GIH WR"]
x

,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD
Brutal Cathar,WR,R,2928,1.413,1934,1.395,2391,62.108,426,65.962,849,67.609,1275,67.059,1223,57.073,9.986
"Dennick, Pious Apprentice",WU,R,4968,2.604,2061,2.542,6192,61.337,1145,68.559,2087,64.494,3232,65.934,3205,57.254,8.680
"Adeline, Resplendent Cathar",W,R,3035,1.516,1906,1.517,2605,61.459,488,66.189,968,65.393,1456,65.659,1249,57.246,8.414
Grafted Identity,U,R,3522,1.755,2483,1.739,2901,61.668,510,63.725,933,66.238,1443,65.350,1477,58.158,7.192
Suspicious Stowaway,UG,R,3483,1.741,2308,1.727,2693,60.527,528,71.023,939,61.874,1467,65.167,1327,56.443,8.724


In [12]:
PANDAS_CACHE[s][f]['']

NameError: name 's' is not defined

In [19]:
s = SETS[0]
f = FORMATS[0]
get_color_metadata(s, f, 'U')

(Top 10 Commons         Color Rarity  # Seen   ALSA  # Picked     ATA   # GP  \
 Organ Hoarder              U      C   74818  4.312     29965   4.736  99337   
 Revenge of the Drowned     U      C   89814  5.111     25838   6.240  88668   
 Falcon Abomination         U      C   75248  4.150     22500   5.101  81468   
 Flip the Switch            U      C  100344  6.107     21013   8.065  62072   
 Shipwreck Sifters          U      C  108457  6.968     19866   9.238  31097   
 Startle                    U      C  103189  6.233     20932   8.434  62962   
 Larder Zombie              U      C  112482  7.724     19722  10.343  32426   
 Baithook Angler            U      C   94550  5.336     24050   6.890  81599   
 Consider                   U      C  105484  6.395     19796   8.794  47766   
 Evolving Wilds                    C   92968  4.820     19033   6.790  85467   
 
 Top 10 Commons           GP WR   # OH   OH WR   # GD   GD WR  # GIH  GIH WR  \
 Organ Hoarder           58.879  27878

In [ ]:
s = SETS[0]
f = FORMATS[0]
stat = 'GP WR'
#get_top(PANDAS_CACHE[s][f][''], stat, n=5, card_colors=None, card_rarity=None)

rarities = ['', 'C', 'U', 'R', 'M']


def get_top_5s():
    top_5_dict = dict()
    colors = ['W', 'U', 'B', 'R', 'G']
    rarity_sets = [('C', 5), ('U', 5), ('RM', 3)]
    
    for c1 in colors:
        for c2 in colors:
            c = WUBRG.get_color_identity(c1 + c2)
            top_5_dict[c] = dict()
            for r in rarity_sets:
                color_filter = c if (len(c) == 1) else None
                top_5_dict[c][r[0]] = get_top(PANDAS_CACHE[s][f][c], stat, n=r[1], card_colors=color_filter, card_rarity=r[0])
                alias = WUBRG.get_color_alias(c)
                top_5_dict[c][r[0]].columns.name = alias
    return top_5_dict

#top_5_dict = get_top_5s()

def top_5_snippet(color, rarity):
    trgt_cols = ['ALSA', 'ATA', '# GP', 'GP WR', '# GIH', 'GIH WR']
    return top_5_dict[color][rarity][trgt_cols]

#top_5_snippet('UR', 'U')

In [ ]:
premier_commons = premier[premier['Rarity'] == 'C']
premier_commons['GP WR'].nlargest(n=5)

In [ ]:
colours = ''
rarity = 'U'
get_trap_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_niche_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_staple_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_dreg_cards('MID', 'PremierDraft', colours, rarity, -0.005)